<h1> NEAT Implementation Practice </h1>
My Raw NEAT implementation, specifically for the cartpole environment.

# Some Notes
Here are some things about which knobs you can turn in each cell:
1. Imports. Don't touch anything there unless you wanna try something out at your own risk.
2. Environment Declarations. Change the envName string if you want to test out other environments, but just note that you may need to change some other parameters in the evaluation cell as well if you do so.
3. Hyperparameters. Change as you please, but I would recommend sticking to the values from the NEAT paper and/or neatevolve.lua.
4. Some functions. I wouldn't touch this.
5. Genome Class. I wouldn't touch this, but please feel free to look at the mess I've made. I have recently moved the crossover and delta functions as static methods to this class.
8. Evaluation. As mentioned in 2), if you change the environment, then you made need to change the evaluateAction method of the genome or some of the target fitness requirements for episode termination.

In [1]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random|
import math
import time # for timing how long it runs for 
import PySimpleGUI as sg
import pickle 
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt

In [3]:
# first define the environment and get the both the action and observation spaces 
envName = 'CartPole-v1' # just need to change this if you wanna try it in environments
env = gym.make(envName)
plt.title(envName + ' Average Fitness across Generations - RNN')
# simple gui stuff
sg.theme = ('DarkAmber')
layout = [  [sg.Text('Trial: '), sg.Text(0, key='TRIAL-NUM', size= (6, 1))],
            [sg.Text('Simulation Info'), sg.Text(''),sg.Text('BestGenome Info')],
            [sg.Text('Generation: '), sg.Text(0, key='GENER-NUM', size = (6,1)), sg.Text('Generation: '), sg.Text(0, key='BEST-GENER-NUM', size = (6,1))],
            [sg.Text('Genome: '), sg.Text(0, key='GENOM-NUM',size = (6,1)), sg.Text('Generation: '), sg.Text(0, key='BEST-GENOM-NUM', size = (6,1))],
            [sg.Text('Species: '), sg.Text(0, key='SPECIES-NUM',size = (6,1)), sg.Text('Species: '), sg.Text(0, key='BEST-SPECIES-NUM',size = (6,1))],
            [sg.Text('Fitness: '), sg.Text(0, key='FITNESS-NUM',size = (6,1)), sg.Text('Fitness: '), sg.Text(0, key='BEST-FITNESS-NUM',size = (6,1))]]



# # print out the observation and action spaces
# print(env.action_space.shape[0]) # number of valid actions we can take 
# print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
# NOTE: changes between Box() and Discrete() depending on environment
outputs = env.action_space.n
inputs = env.observation_space.shape[0] + 1


In [4]:
# hyperparameters, according to section 4.1 Parameter Settings (changed some parameters for myself)
deltaThreshold = 1.0

c1 = c2 = 2.0
c3 = 0.4
perturbVal = 0.025


numUntilStagnant = 15


genomeMutationChance = 0.25 # neatevolve.lua has this pretty low 
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75 
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.01 
smallPopulationNewNodeChance = 0.05
largePopulationNewNodeChance = 0.50 # using these right now 
smallPopulationNewConnectionChance = 0.03
largePopulationNewConnectionChance = 0.3 # using these right now 

In [5]:
# activation functions 
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [6]:
# genome class 
class Genome:
    # initialize class variables 
    globalInnovationNumber = 1
    historyMaxLength = 10
    def __init__(self, inputs=1, outputs=1):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.inputNodes = []
        self.outputNodes = []
        self.hiddenNodes = []
        self.connectGenes = {}
        self.fitness = 0
        self.sharedFitness = 0
        self.species = 0
        self.inputCount = inputs
        self.outputCount = outputs
        self.mutationSuccess = False
        
    def setGenes(self, nodeGenes, connectGenes):
        self.nodeGenes = nodeGenes.copy()
        self.connectGenes = connectGenes.copy()
        
    @staticmethod
    def copyGenome(genome):
        g = Genome(genome.inputCount, genome.outputCount)
        g.setGenes(genome.nodeGenes, genome.connectGenes)
        g.inputNodes = genome.inputNodes.copy()
        g.hiddenNodes = genome.hiddenNodes.copy()
        g.outputNodes = genome.outputNodes.copy()
        g.clearNodeValues()
        g.clearHistoryLists()
        return g
    
    @staticmethod
    def resetGlobalInnovationNumber():
        Genome.globalInnovationNumber = 1
        # print('Genome global innovation number reset back to: {}'.format(Genome.globalInnovationNumber))
    
    @staticmethod
    def crossover(genome1, genome2):
        if (genome2.fitness > genome1.fitness):
            temp = genome1
            genome1 = genome2
            genome2 = temp

        g = Genome(genome1.inputCount, genome1.outputCount)
        g1GenomeList = {}
        for connection in genome1.connectGenes:
            #print(genome1.connectGenes[connection])
            g1GenomeList[genome1.connectGenes[connection]['historicalMarker']]={'connection':connection, 'weight': genome1.connectGenes[connection]['weight'],'isExpressed': genome1.connectGenes[connection]['isExpressed']}
        g2GenomeList = {}
        for connection in genome2.connectGenes:
            g2GenomeList[genome2.connectGenes[connection]['historicalMarker']]={'connection':connection, 'weight': genome2.connectGenes[connection]['weight'],'isExpressed': genome2.connectGenes[connection]['isExpressed']}

        # when crossing over, the genes in both genomes with the same innovation numbers are lined up
        # in composing the offspring, genes are randomly chosen from either parent at matching genes,
        # whereas all excess or disjoint genes are always included from the more fit parent 
        for i in range(max(len(g1GenomeList), len(g2GenomeList))):
            if (i+1 in g1GenomeList and i+1 in g2GenomeList):
                if (random.random() <= 50): # inherit from genome1
                    # print('inherited from parent 1')
                    g.connectGenes[g1GenomeList[i+1]['connection']] = {'historicalMarker':i+1, 'weight':g1GenomeList[i+1]['weight'], 'isExpressed':g1GenomeList[i+1]['isExpressed']}
                    # [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
                else: # inherit from genome2 
                    # print('inherited from parent 2')
                    g.connectGenes[g2GenomeList[i+1]['connection']] = {'historicalMarker':i+1, 'weight':g2GenomeList[i+1]['weight'], 'isExpressed':g2GenomeList[i+1]['isExpressed']}
                    # [i+1, g2GenomeList[i+1][1], g2GenomeList[i+1][2]]
                if (g1GenomeList[i+1]['isExpressed'] == False or g1GenomeList[i+1]['isExpressed'] == False):
                    if (random.random() <= disableGeneChance):
                        # print('inherited disabled gene')
                        g.connectGenes[g1GenomeList[i+1]['connection']]['isExpressed'] = False
                    else:
                        # print('re-enabled inherited gene')
                        g.connectGenes[g1GenomeList[i+1]['connection']]['isExpressed'] = True
            if (i+1 in g1GenomeList and i+1 not in g2GenomeList):
                g.connectGenes[g1GenomeList[i+1]['connection']] = {'historicalMarker':i+1, 'weight':g1GenomeList[i+1]['weight'], 'isExpressed':g1GenomeList[i+1]['isExpressed']}
                    # [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
            # insert the node genes 

        for c in g.connectGenes:
            if c[0] in g1.nodeGenes:
                g.nodeGenes[c[0]] = g1.nodeGenes[c[0]]
            elif c[0] in g2.nodeGenes:
                g.nodeGenes[c[0]] = g2.nodeGenes[c[0]]
                
            if (g.nodeGenes[c[0]]['type'] == 'input'):
                g.inputNodes.append(c[0])
            elif (g.nodeGenes[c[0]]['type'] == 'output'):
                g.outputNodes.append(c[0])
            elif (g.nodeGenes[c[0]]['type'] == 'hidden'):
                g.hiddenNodes.append(c[0])

            if c[1] in g1.nodeGenes:
                g.nodeGenes[c[1]] = g1.nodeGenes[c[1]]
            elif c[1] in g2.nodeGenes:
                g.nodeGenes[c[1]] = g2.nodeGenes[c[1]]
                
            if (g.nodeGenes[c[1]]['type'] == 'hidden'):
                g.hiddenNodes.append(c[1])
                
            if (g.nodeGenes[c[1]]['type'] == 'input'):
                g.inputNodes.append(c[1])
            elif (g.nodeGenes[c[1]]['type'] == 'output'):
                g.outputNodes.append(c[1])
            elif (g.nodeGenes[c[1]]['type'] == 'hidden'):
                g.hiddenNodes.append(c[1])
        
        
        g.inputNodes = list(set(g.inputNodes))
        g.outputNodes = list(set(g.outputNodes))
        g.hiddenNodes = list(set(g.hiddenNodes))
        
        # also need to replace the adjacent lists now 
        g.resetAdjacentLists()
        for c in g.connectGenes:
            g.nodeGenes[c[1]]['adjacentNodes'].append(c[0]) 
            
        for n in g.nodeGenes:
            g.nodeGenes[n]['adjacentNodes'] = list(set(g.nodeGenes[n]['adjacentNodes']))
            
        g.clearNodeValues()
        g.clearHistoryLists()
        return g
    
    @staticmethod
    def delta(genome1, genome2, c1=0.1, c2=0.1, c3=0.1):
        if (not genome1.connectGenes or not genome2.connectGenes):
            return 0
        connectionMismatch = []
        for connection1 in genome1.connectGenes:
            if (connection1 not in genome2.connectGenes):
                connectionMismatch.append(connection1)
        for connection2 in genome2.connectGenes:
            if (connection2 not in genome1.connectGenes):
                connectionMismatch.append(connection2)
        diffsum = 0
        num = 0
        for connection1 in genome1.connectGenes:
            if (connection1 not in connectionMismatch):
                diffsum += abs(genome1.connectGenes[connection1]['weight'] -
                               genome2.connectGenes[connection1]['weight'])
                num += 1
        averageWeightDifference = diffsum / num
        #print(averageWeightDifference)
        N = 0
        if len(genome1.connectGenes) >= len(genome2.connectGenes):
            N = len(genome1.connectGenes)
        else:
            N = len(genome2.connectGenes)
        print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
        return ((len(connectionMismatch) *
                 (c1 + c2)) / N) + (c3 * averageWeightDifference)
    
    def clearGenome(self):
        self.globalInnovation = 1
        self.nodeGenes.clear()
        self.connectGenes.clear()
        
    def clearHistoryLists(self):
        for node in self.nodeGenes:
            self.nodeGenes[node]['valueHistory'].clear()
        
    def clearNodeValues(self):
        for node in self.nodeGenes:
            self.nodeGenes[node]['value'] = -2.0
            
    def initEmptyGenome(self):
        self.clearGenome()
        for i in range(self.inputCount):
            self.insertNode('input')
        for i in range(self.outputCount):
            self.insertNode('output')
            
    def resetFitness(self):
        self.fitness = 0
        self.sharedFitness = 0
        
    def resetAdjacentLists(self):
        for n in self.nodeGenes:
            self.nodeGenes[n]['adjacentNodes'] = []
        
    def initRandomGenome(self):
        self.initEmptyGenome()
        # brute force connect each new input node to each output node 
        for node in self.nodeGenes:
            if self.nodeGenes[node]['type'] == 'input':
                for otherNode in self.nodeGenes:
                    if self.nodeGenes[otherNode]['type'] == 'output':
                        self.insertConnection(outNode=node, inNode=otherNode, weight=random.uniform(-1, 1))
             
    def insertNode(self, nodeType='input', value = 0.0):
        self.nodeGenes[len(self.nodeGenes)+1]={'type': nodeType, 'value': value, 'valueHistory':[], 'adjacentNodes':[]}
        if (nodeType == 'input'):
            self.inputNodes.append(len(self.nodeGenes))
        elif (nodeType == 'output'):
            self.outputNodes.append(len(self.nodeGenes))
        else:
            self.hiddenNodes.append(len(self.nodeGenes))
 
    def insertConnection(self, outNode, inNode, weight = 0.5, isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) not in self.connectGenes and inNode != 1: # check if the connection already exists, and do not feed into the bias cell
                self.connectGenes[(outNode, inNode)] = {'historicalMarker': Genome.globalInnovationNumber, 'weight': weight, 'isExpressed': isExpressed}
                self.nodeGenes[inNode]['adjacentNodes'].append(outNode)
                Genome.globalInnovationNumber += 1
                self.mutationSuccess = True
            
    def checkConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                return self.connectGenes[(outNode, inNode)]['isExpressed']
            else:
                print('Could not find the connection')
        else:
            print('Could not find node(s). Did not disable.')
            
    def disableConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.connectGenes[(outNode, inNode)]['isExpressed'] = False
            else:
                print('Could not find connection to disable. Did not disable.')
        else: 
            print('Could not find node(s). Did not disable.')
    
    def mutateAddConnection(self, outNode, inNode):
        self.insertConnection(outNode, inNode, random.uniform(-1, 1), True)
            
    def mutateAddRandomConnection(self):
        timesAttemptedToConnected = 0
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            self.mutateAddConnection(random.choice(list(self.nodeGenes)),random.choice(list(self.nodeGenes)))
            timesAttemptedToConnected +=1
            if (timesAttemptedToConnected > 100):
                print('Attempted to connect too many times. probably already full')
                return
        self.mutationSuccess = False
        
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.insertNode('hidden', 0.0)
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)]['weight']
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
                self.mutationSuccess = True
                print('added hidden node')
            else:
                print('Error! No connection found.')
        else:
            print('Could not find node(s). Did not mutate.')
    
    def mutateAddRandomNode(self):
        if (not self.connectGenes): # need to debug this 
            print('connectGenes is empty. can\'t mutateAddRandomNode')
            return
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            randomTuple = random.choice(list(self.connectGenes))
            self.mutateAddNode(randomTuple[0],randomTuple[1])
        self.mutationSuccess = False
        
    def recurrentConnection(self, originalNode, currentNode):
        if (originalNode == currentNode):
            return True
        if (currentNode in self.inputNodes):
            return False 
        for node in self.nodeGenes[currentNode]['adjacentNodes']:
            if (node != currentNode): # not interested in self-loops
                if(self.recurrentConnection(originalNode, node)):
                    return True
        return False

    # implemented first for the non-recurrent case 
    def nodeRecur(self, nodeKey):
        nodeSum = 0
        for n in self.nodeGenes[nodeKey]['adjacentNodes']:
            if (self.nodeGenes[n]['type'] == 'input' or self.nodeGenes[n]['value'] != -2): # input
                print('first-case')
                nodeSum += self.nodeGenes[n]['value'] * self.connectGenes[(n, nodeKey)]['weight']
            elif (self.recurrentConnection(nodeKey, n)): # TODO: implement
                print('detected recurrent connection (second case)')
                nodeSum += self.connectGenes[(n, nodeKey)]['weight'] * sum(self.nodeGenes[n]['valueHistory'])
            elif (self.nodeGenes[n]['value'] == -2): # unevaluted (recursive case)
                print('third-case')
                nodeSum += self.nodeRecur(n) * self.connectGenes[(n, nodeKey)]['weight']
        
        if (self.nodeGenes[nodeKey]['type'] == 'output'):
            print('output value: {}'.format(np.exp(nodeSum)))
            return np.exp(nodeSum)
        else:
            return steepsigmoid(nodeSum)
        # TODO: Implement
    
    # NOTE: RECURRENT
    def evaluateAction(self, observation):
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1]['value'] = 1.0 # bias node 
            
            # load the observation into the input nodes
            for i in range(self.inputCount-1):
                self.nodeGenes[i+2]['value'] = observation[i]
                
                # if there are any recurrent connections, grab their history 
                for n in self.nodeGenes[i+2]['adjacentNodes']:
                    self.nodeGenes[i+2]['value'] += self.connectGenes[(n, i+2)]['weight'] * sum(self.nodeGenes[n]['valueHistory'])
            
            # process all hidden nodes 
            for h in self.hiddenNodes:
                print('node:{}'.format(h))
                if (self.nodeGenes[h]['value'] == -2.0):
                    self.nodeGenes[h]['value'] = self.nodeRecur(h)
            
            
            # at this point, every node aside from the output nodes should be filled. calculate the softmaxsum
            print('calculating output node values...')
            softmaxsum = 0
            for o in self.outputNodes:
                self.nodeGenes[o]['value'] = self.nodeRecur(o)
                print(self.nodeGenes[o]['value'])
                softmaxsum += self.nodeGenes[o]['value']
                
            output = []
            # before we evaluate, go back and update the history of each node
            for node in self.nodeGenes:
                if self.nodeGenes[node]['type'] == 'output':
                    historyValue = self.nodeGenes[node]['value'] / softmaxsum
                    output.append(historyValue)
                else:
                    historyValue = self.nodeGenes[node]['value']
                # check if the history is full
                if (len(self.nodeGenes[node]['valueHistory']) == Genome.historyMaxLength): # if it is, pop the back and insert into the front
                    self.nodeGenes[node]['valueHistory'].pop()
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
                else:
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
            return output.index(max(output))
        
    def evaluateActionBox(self, observation):
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1]['value'] = 1.0 # bias node 
            
            # load the observation into the input nodes
            for i in range(self.inputCount-1):
                self.nodeGenes[i+2]['value'] = observation[i]
                
                # if there are any recurrent connections, grab their history 
                for n in self.nodeGenes[i+2]['adjacentNodes']:
                    self.nodeGenes[i+2]['value'] += self.connectGenes[(n, i+2)].weight * sum(self.nodeGenes[n]['valueHistory'])
            
            # process all hidden nodes 
            for h in self.hiddenNodes:
                if (self.nodeGenes[h]['value'] == -2.0):
                    self.nodeGenes[h]['value'] = self.nodeRecur(h)
            
            
            # at this point, every node aside from the output nodes should be filled. calculate the softmaxsum
            softmaxsum = 0
            for o in self.outputNodes:
                self.nodeGenes[o]['value'] = self.nodeRecur(o)
                softmaxsum += self.nodeGenes[o]['value']
                
            output = []
            # before we evaluate, go back and update the history of each node
            for node in self.nodeGenes:
                if self.nodeGenes[node]['type'] == 'output':
                    historyValue = self.nodeGenes[node]['value'] / softmaxsum
                    output.append(historyValue)
                else:
                    historyValue = self.nodeGenes[node]['value']
                # check if the history is full
                if (len(self.nodeGenes[node]['valueHistory']) == Genome.historyMaxLength): # if it is, pop the back and insert into the front
                    self.nodeGenes[node]['valueHistory'].pop()
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
                else:
                    self.nodeGenes[node]['valueHistory'].insert(0, historyValue)
            return output
    
    def evaluateSharedFitness(self, speciesCount):
        self.sharedFitness = self.fitness / speciesCount
    
    def printNodeGenes(self):
        print(self.nodeGenes)
        
    def printNodeGeneCount(self):
        print(len(self.nodeGenes))
        
    def printConnectGenes(self):
        print(self.connectGenes)
    
    def printConnectGeneCount(self):
        print(len(self.connectGenes))
    
    def printActiveConnectGenes(self):
        for connection in self.connectGenes:
            if (self.connectGenes[connection]['isExpressed']):
                print(connection, self.connectGenes[connection]['weight'])

In [7]:
g1 = Genome(inputs, outputs)
Genome.resetGlobalInnovationNumber()
g1.initRandomGenome()
g1.fitness = 100
g1.insertConnection(6, 1)
g1.mutateAddNode(1, 6)
print(g1.inputNodes)
print(g1.outputNodes)

g2 = Genome(inputs, outputs)
Genome.resetGlobalInnovationNumber()
g2.initRandomGenome()
g2.inputNodes
print(g2.inputNodes)
print(g2.outputNodes)

g3 = Genome.crossover(g1, g2)
print(g3.inputNodes)
print(g3.outputNodes)
print(g3.hiddenNodes)

added hidden node
[1, 2, 3, 4, 5]
[6, 7]
[1, 2, 3, 4, 5]
[6, 7]
[1, 2, 3, 4, 5]
[6, 7]
[8]


In [8]:
print('g1 stuff')
g1.printConnectGenes()
print(' ')
g1.printNodeGenes()
print(' ')
print('g2 stuff')
g2.printConnectGenes()
print(' ')
g2.printNodeGenes()
print(' ')
print('g3 stuff')
g3.printConnectGenes()
print(' ')
g3.printNodeGenes()
print(g3.inputNodes)

g1 stuff
{(1, 6): {'historicalMarker': 1, 'weight': -0.999375199975223, 'isExpressed': False}, (1, 7): {'historicalMarker': 2, 'weight': 0.8684320390629077, 'isExpressed': True}, (2, 6): {'historicalMarker': 3, 'weight': -0.5365730527083636, 'isExpressed': True}, (2, 7): {'historicalMarker': 4, 'weight': 0.7202684598548597, 'isExpressed': True}, (3, 6): {'historicalMarker': 5, 'weight': -0.11151113838916582, 'isExpressed': True}, (3, 7): {'historicalMarker': 6, 'weight': -0.2936230172666441, 'isExpressed': True}, (4, 6): {'historicalMarker': 7, 'weight': -0.9264872344209911, 'isExpressed': True}, (4, 7): {'historicalMarker': 8, 'weight': 0.03158477582625907, 'isExpressed': True}, (5, 6): {'historicalMarker': 9, 'weight': 0.5078200008450398, 'isExpressed': True}, (5, 7): {'historicalMarker': 10, 'weight': 0.14298499149240396, 'isExpressed': True}, (1, 8): {'historicalMarker': 11, 'weight': 1.0, 'isExpressed': True}, (8, 6): {'historicalMarker': 12, 'weight': -0.999375199975223, 'isExpre

> connection weights mutate as in any NE system, with each connection either perturbed or not at each generation 

In [9]:
max_generations = 10
population_limit = 10
championThresholdCount = 5

inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.n

bestGenomeDictionary = {}

targetFitness = 500
maxTestTrials = 1
targetAverageFitness = 495.5

window = sg.Window(envName, layout, finalize=True)
start_time = time.time()

globalEvaluationCounter = 0
solvedCounter = 0

# NOTE: input/output changes depending on environment !

for q in range(maxTestTrials):
    hl, = plt.plot([],[])
    window['TRIAL-NUM'].update(q+1)
    species = {}
    pop = {}
    solved = False
    trialLoopCounter = 0
    bestGenome = None
    highestFitness = float("-inf")
    for generation in range(max_generations):
        generationFitness = 0
        print('Beginning Generation #{}'.format(generation+1))
        window['GENER-NUM'].update(generation+1)
        if (generation == 0 and not pop): # start out with a uniform population of networks with zero hidden networks
            for genome in range(population_limit):
                g = Genome(inputs, outputs)
                Genome.resetGlobalInnovationNumber()
                g.initRandomGenome()
                pop[genome+1] = g

        # in each generation, genomes are sequentially placed into species 

        fitness = []

        for genome in pop:
            print('Beginning evaluation on genome #{}'.format(genome))
            window['GENOM-NUM'].update(genome)
            if (not species):
                print('creating new species:{}'.format(len(species)+1))
                species[len(species)+1] = {'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter' : 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
                pop[genome].species = len(species)
                print(len(species))
            else:
                for og in species:
                    if (len(species[og]['list']) == 0):
                        print('inserting into species#{}'.format(og))
                        species[og]['list'].append(pop[genome])
                        pop[genome].species = og
                        break
                    else:
                        if (Genome.delta(pop[genome], species[og]['list'][0], c1, c2, c3) < deltaThreshold): # just compare with the first species
                            print('inserting into species#{}'.format(og))
                            species[og]['list'].append(pop[genome])
                            pop[genome].species = og
                            break
                        else:
                            if (og == len(species)):
                                print('creating new species:{}'.format(len(species)+1))
                                species[len(species)+1] ={'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
                                pop[genome].species = len(species)
                                print(len(species))
                                break

            window['SPECIES-NUM'].update(pop[genome].species)
            observation = env.reset()
            pop[genome].resetFitness()
            # evalute the genome's performance # SDHJKLSDHLSJKDHLSDHJKLHDSKLDHJ
            t = 0
            trialLoopCounter += 1
            while(True):
                t+=1
                env.render()
                action = pop[genome].evaluateAction(observation)
                observation, reward, done, info = env.step(action)
                pop[genome].fitness += reward
                window['FITNESS-NUM'].update(pop[genome].fitness)
                window.finalize()
                if done:
                    print("Episode finished after {} timesteps".format(t+1))
                    print("Genome fitness: {}".format(pop[genome].fitness))
                    fitness.append(pop[genome].fitness)
                    generationFitness+=pop[genome].fitness
                    if pop[genome].fitness > species[pop[genome].species]['maxFitness']:
                        species[pop[genome].species]['maxFitness'] = pop[genome].fitness
                        species[pop[genome].species]['stagnantCounter'] = 0
                        species[pop[genome].species]['isStagnant'] = False
                    if pop[genome].fitness > highestFitness:
                        bestGenome = pop[genome]
                        highestFitness = pop[genome].fitness
                        # update the window values 
                        window['BEST-GENER-NUM'].update(generation+1)
                        window['BEST-GENOM-NUM'].update(genome)
                        window['BEST-SPECIES-NUM'].update(pop[genome].species)
                        window['BEST-FITNESS-NUM'].update(pop[genome].fitness)
                        window.finalize()
                    if pop[genome].fitness >= targetFitness: # changes depending on environment !
                        print('Found an optimal genome. testing it...')
                        testBenchTotalFitness = 0
                        for i in range(maxTestTrials):
                            m = 0
                            pop[genome].resetFitness()
                            observation = env.reset()
                            while(True):
                                m+=1
                                env.render()
                                action = pop[genome].evaluateAction(observation)
                                observation, reward, done, info = env.step(action)
                                pop[genome].fitness += reward
                                window['FITNESS-NUM'].update(pop[genome].fitness)
                                window.finalize()
                                if done:
                                    testBenchTotalFitness+=pop[genome].fitness
                                    print("Genome fitness: {}".format(pop[genome].fitness))
                                    break
                        print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/maxTestTrials))
                        if (testBenchTotalFitness/maxTestTrials >= targetAverageFitness):
                            print('found solution genome')
                            solved = True
                            solvedCounter +=1
                            bestGenomeDictionary[len(bestGenomeDictionary)+1] = pop[genome]
                    break
            if (solved):
                break
        if (solved):
            break
            
        # update the plot of the average fitness across generations 
        hl.set_xdata(np.append(hl.get_xdata(), generation))
        hl.set_ydata(np.append(hl.get_ydata(), generationFitness/population_limit))
        plt.plot(hl.get_xdata(), hl.get_ydata())
        plt.draw()
        print('Number of species at generation {}: {}'.format(generation+1, len(species)))
        # print('highest achieved score in this generation {}'.format(max(fitness)))

        # print(species[1])
        adjustedFitness = []
        totalAdjustedFitness = 0.0
        for genome in pop:
            pop[genome].evaluateSharedFitness(len(species[pop[genome].species]))
            adjustedFitness.append(pop[genome].sharedFitness)
            species[pop[genome].species]['summedSharedFitness'] += pop[genome].sharedFitness
            if pop[genome].sharedFitness > species[pop[genome].species]['maxAdjustedFitness']:
                species[pop[genome].species]['maxAdjustedFitness'] = pop[genome].sharedFitness
                species[pop[genome].species]['bestGenome'] = pop[genome]

        for s in species:
            totalAdjustedFitness += species[s]['summedSharedFitness']
            print('Maximum adjusted fitness in species {}: {}'.format(s, species[s]['maxAdjustedFitness']))
            print('Summed adjusted fitness in species {}: {}'.format(s, species[s]['summedSharedFitness']))

        # print('Total Adjusted Fitness across all species: {}'.format(totalAdjustedFitness))

        newPop = {}

        # the champion of each species with more than 5 networks will be copied into the next generation unchanged
        i = 0
        for s in species:
            if len(species[s]['list']) > championThresholdCount:
                newPop[i+1] = Genome.copyGenome(species[s]['bestGenome'])
                i += 1

        print('New Pop count after copying champions only: {}'.format(len(newPop)))
        print('i\'s value after inserting champions: {}'.format(i))


        # getting the proportion of offspring they are allowed
        totalBreedCount = 0
        for s in species:
            if species[s]['isStagnant']:
                print('species #{} is stagnant'.format(s))
                species[s]['breedAmountAllowed'] = 0
            else:
                species[s]['breedAmountAllowed'] = np.floor((species[s]['summedSharedFitness']/totalAdjustedFitness)*(population_limit-i)).astype(int)
                totalBreedCount += species[s]['breedAmountAllowed']


        leftovers = population_limit - i - totalBreedCount
        print('leftovers:{}'.format(leftovers))
        
        # get rid of the "lowest performing members" of the population
#         amountToCull = np.floor(.75*len(pop)).astype(int)
#         print('Amount to cull: {}'.format(amountToCull))
#         # print(sorted(pop.items(), key=lambda x: x[1].fitness))
#         for b in range(amountToCull):
#             tempTupleList = sorted(pop.items(), key=lambda x:x[1].fitness, reverse = True)
#             tempGenomeKey = tempTupleList.pop()[0]
#             tempGenome = pop.pop(tempGenomeKey)
#             species[tempGenome.species]['list'].remove(tempGenome)
#             print('Removed Genome # {} from species # {}, with a fitness of {}, from the population'.format(tempGenomeKey, tempGenome.species, tempGenome.fitness))


        for s in species:
            amountToCull = np.floor(.75*len(species[s]['list'])).astype(int)
            species[s]['list'].sort(reverse = True, key=lambda genome: genome.fitness)
            for ge in  species[s]['list']:
                print(ge.fitness)
            for b in range(amountToCull):
                tempGenome = species[s]['list'].pop() 
                # print(tempGenome.fitness)

        # now for each species breed !
        for s in species:
            if (len(species[s]['list']) >= 1):
                print('species {} is allowed to have {} offspring'.format(s, species[s]['breedAmountAllowed']))
                for j in range(species[s]['breedAmountAllowed']):
                    print('making offspring #{}'.format(i+1))
                    if (random.random() <= mutateNoCrossoverChance or len(species[s]['list'])==1): # either the genome is a crossover or just the same genome 
                        print('Chose no crossover')
                        g = Genome.copyGenome(random.choice(species[s]['list']))
                    else:
                        print('Chose crossover')
                        g1 = random.choice(species[s]['list'])
                        g2 = random.choice(species[s]['list'])
                        g = Genome.crossover(g1, g2)

                    # add a chance to mutate the connections
                    if (g.connectGenes):
                        if (random.random() <= genomeMutationChance):
                            print('mutation genome connections !')
                            for connection in g.connectGenes:
                                if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                                    g.connectGenes[connection]['weight'] += random.uniform(-perturbVal, perturbVal)
                                else:
                                    g.connectGenes[connection]['weight'] = random.uniform(-1.0, 1.0) # set a new value for 


                    # chance to add new connections/nodes
                    if (random.random() <= largePopulationNewNodeChance):
                        print('trying to add new node')
                        g.mutateAddRandomNode()
                    if (random.random() <= largePopulationNewConnectionChance):
                        print('trying to add new connection')
                        g.mutateAddRandomConnection()

                    newPop[i+1] = g
                    i+=1
            else:
                print('could not breed species{}, list is empty !'.format(s))
        # -----------------------------------------------------------------------------------------------------        

        if (leftovers > 0): 
            print('now for the leftovers:')
        for k in range(leftovers): # breeding the leftover genomes that were not calculated through floor
            if (random.random() <= interspeciesMatingRate): # very small chance
                print('interspecies')
                speciesKey1 = random.choice(list(species))
                speciesKey2 = random.choice(list(species))
                if (len(species[speciesKey1]['list']) < 1 or len(species[speciesKey2]['list']) < 1):
                    print('either species does not have enough to breed. skipping.')
                    k -= 1
                    continue
                g1 = random.choice(species[speciesKey1]['list'])
                g2 = random.choice(species[speciesKey2]['list'])
                g = Genome.crossover(g1, g2)
                if (g.connectGenes):
                    if (random.random() <= genomeMutationChance):
                        print('mutation genome connections !')
                        for connection in g.connectGenes:
                            if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                                g.connectGenes[connection]['weight'] += random.uniform(-perturbVal, perturbVal)
                            else:
                                g.connectGenes[connection]['weight'] = random.uniform(-1.0, 1.0) # set a new value for 


                # chance to add new connections/nodes
                if (random.random() <= largePopulationNewNodeChance):
                    print('trying to add new node')
                    g.mutateAddRandomNode()
                if (random.random() <= largePopulationNewConnectionChance):
                    print('trying to add new connection')
                    g.mutateAddRandomConnection()


                newPop[i+1] = g
                i+=1
            else: # else just find a random species and mate from there 
                print('random species mating')
                speciesKey = random.choice(list(species))
                if (len(species[speciesKey]['list']) < 1):
                    print('species does not have enough to breed. skipping.')
                    k -= 1
                    continue
                if (random.random() <= mutateNoCrossoverChance or len(species[speciesKey]['list']) < 2): # either the genome is a crossover or just the same genome 
                    print('Chose no crossover')
                    g = Genome.copyGenome(random.choice(species[speciesKey]['list']))
                else:
                    print('Chose crossover')
                    g1 = random.choice(species[speciesKey]['list'])
                    g2 = random.choice(species[speciesKey]['list'])
                    g = Genome.crossover(g1, g2)

                # add a chance to mutate the connections
                if (g.connectGenes):
                    if (random.random() <= genomeMutationChance):
                        print('mutation genome connections !')
                        for connection in g.connectGenes:
                            if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                                g.connectGenes[connection]['weight'] += random.uniform(-perturbVal, perturbVal)
                            else:
                                g.connectGenes[connection]['weight'] = random.uniform(-1.0, 1.0) # set a new value for 


                # chance to add new connections/nodes
                if (random.random() <= largePopulationNewNodeChance):
                    print('trying to add new node')
                    g.mutateAddRandomNode()
                if (random.random() <= largePopulationNewConnectionChance):
                    print('trying to add new connection')
                    g.mutateAddRandomConnection()


                newPop[i+1] = g
                i+=1


        print('Final value of i: {}'.format(i))

        # ------------------------------------------------------------------------------------------

        # each existing species is represented by a random genome inside the species from the previous generation 
        if (generation >= 0):
            for s in species:
                if (len(species[s]['list']) == 0):
                    print('Species extinct! Emptying it and starting a new one...')
                    species[s] = {'summedSharedFitness': 0.0, 'maxFitness': 0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant': False, 'list' : []}
                    species[s]['bestGenome'] = None
                else:
                    species[s] = {'summedSharedFitness': 0.0, 'maxFitness': 0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': species[s]['stagnantCounter']+1, 'isStagnant': False, 'list' : [random.choice(species[s]['list'])]}
                    species[s]['bestGenome'] = species[s]['list'][0]
                if species[s]['stagnantCounter'] >= numUntilStagnant:  # if the maximum fitness of a species did not improve in 15 generations, the networks in the stagnant species were not allowed to reproduce 
                    species[s]['isStagnant'] = True

        pop.clear()
        pop = newPop.copy()


    print('Total number of evaluations for this trial: {}'.format(trialLoopCounter))
    globalEvaluationCounter += trialLoopCounter 

print("--- %s seconds ---" % (time.time() - start_time))
print('{} solved out of {} trials'.format(solvedCounter, maxTestTrials))    
print('Total Number of evaluations overall {}'.format(globalEvaluationCounter))
print('Average evaluation count per trial: {}'.format(globalEvaluationCounter / maxTestTrials))
window.close()
env.close()
# saving contents of population into pickle when it is finished 
with open('data/pop.dictionary', 'wb') as config_dictionary_file:
    pickle.dump(pop, config_dictionary_file)
    
with open('data/bestGenome_cartpole.dictionary', 'wb') as dictionary_file:
    pickle.dump(bestGenomeDictionary, dictionary_file)

Beginning Generation #1
Beginning evaluation on genome #1
creating new species:1
1
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 2.528993625232552
2.528993625232552
first-case
first-case
first-case
first-case
first-case
output value: 0.7695728026122574
0.7695728026122574
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.6561320229190466
1.6561320229190466
first-case
first-case
first-case
first-case
first-case
output value: 0.8194950010779778
0.8194950010779778
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.080572443137721
1.080572443137721
first-case
first-case
first-case
first-case
first-case
output value: 0.869910008557001
0.869910008557001
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.7014152810177525
0.7014152810177525
first-case
first-case
first-case
first

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.8141904430898368
0.8141904430898368
first-case
first-case
first-case
first-case
first-case
output value: 0.844664376524836
0.844664376524836
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.1023014953029746
1.1023014953029746
first-case
first-case
first-case
first-case
first-case
output value: 0.8826918086104506
0.8826918086104506
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.8435136437153109
0.8435136437153109
first-case
first-case
first-case
first-case
first-case
output value: 0.8362745346714834
0.8362745346714834
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.6485333966290993
0.6485333966290993
first-case
first-case
first-case
first-case
first-case
output value: 0.7903997382627443
0.7903997382627443
calculatin

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.7072921766598459
0.7072921766598459
first-case
first-case
first-case
first-case
first-case
output value: 0.5871824761333307
0.5871824761333307
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.5504971681335954
0.5504971681335954
first-case
first-case
first-case
first-case
first-case
output value: 0.5631096376459815
0.5631096376459815
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.7584587610991872
0.7584587610991872
first-case
first-case
first-case
first-case
first-case
output value: 0.5960638333126369
0.5960638333126369
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.5889702762993905
0.5889702762993905
first-case
first-case
first-case
first-case
first-case
output value: 0.5726952710697434
0.5726952710697434
calculat

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.5124240535692978
0.5124240535692978
first-case
first-case
first-case
first-case
first-case
output value: 2.256140014646772
2.256140014646772
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.56170491575176
0.56170491575176
first-case
first-case
first-case
first-case
first-case
output value: 2.7551224398598344
2.7551224398598344
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.6153307494108302
0.6153307494108302
first-case
first-case
first-case
first-case
first-case
output value: 3.3893332745719986
3.3893332745719986
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.6734806170166245
0.6734806170166245
first-case
first-case
first-case
first-case
first-case
output value: 4.207223166409544
4.207223166409544
calculating outp

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.6234090004998336
1.6234090004998336
first-case
first-case
first-case
first-case
first-case
output value: 1.3810643583848228
1.3810643583848228
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.196148728130781
1.196148728130781
first-case
first-case
first-case
first-case
first-case
output value: 1.5662887017213585
1.5662887017213585
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.6099548374683101
1.6099548374683101
first-case
first-case
first-case
first-case
first-case
output value: 1.3880983180851643
1.3880983180851643
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.1860407310107277
1.1860407310107277
first-case
first-case
first-case
first-case
first-case
output value: 1.5745588911424673
1.5745588911424673
calculatin

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.3350759728586135
1.3350759728586135
first-case
first-case
first-case
first-case
first-case
output value: 1.4653910024411938
1.4653910024411938
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.8001006223764717
1.8001006223764717
first-case
first-case
first-case
first-case
first-case
output value: 1.2876812146531897
1.2876812146531897
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.3290370095655963
1.3290370095655963
first-case
first-case
first-case
first-case
first-case
output value: 1.447338890430787
1.447338890430787
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.7932821447090785
1.7932821447090785
first-case
first-case
first-case
first-case
first-case
output value: 1.2703725074006285
1.2703725074006285
calculatin

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.8538250541382307
0.8538250541382307
first-case
first-case
first-case
first-case
first-case
output value: 1.1999202446069464
1.1999202446069464
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.169102279151089
1.169102279151089
first-case
first-case
first-case
first-case
first-case
output value: 1.0412812618891378
1.0412812618891378
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 0.8776036501501583
0.8776036501501583
first-case
first-case
first-case
first-case
first-case
output value: 1.1540546680651322
1.1540546680651322
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.2027089916952245
1.2027089916952245
first-case
first-case
first-case
first-case
first-case
output value: 1.0005532795181562
1.0005532795181562
calculatin

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 2.640694395964963
2.640694395964963
first-case
first-case
first-case
first-case
first-case
output value: 0.6265158601169589
0.6265158601169589
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 3.044794485113817
3.044794485113817
first-case
first-case
first-case
first-case
first-case
output value: 0.4796814418197718
0.4796814418197718
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 3.5289596647345984
3.5289596647345984
first-case
first-case
first-case
first-case
first-case
output value: 0.36521120437943305
0.36521120437943305
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 4.117456043696305
4.117456043696305
first-case
first-case
first-case
first-case
first-case
output value: 0.27629118985133594
0.27629118985133594
calculatin

calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 2.3230951385853644
2.3230951385853644
first-case
first-case
first-case
first-case
first-case
output value: 1.533570605036518
1.533570605036518
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.608459642082815
1.608459642082815
first-case
first-case
first-case
first-case
first-case
output value: 1.6672453089083803
1.6672453089083803
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 2.409197817802356
2.409197817802356
first-case
first-case
first-case
first-case
first-case
output value: 1.541770415324407
1.541770415324407
calculating output node values...
first-case
first-case
first-case
first-case
first-case
output value: 1.6738847212637367
1.6738847212637367
first-case
first-case
first-case
first-case
first-case
output value: 1.676886795158163
1.676886795158163
calculating output

KeyError: 8

In [10]:
pop[1].printNodeGenes()
print('')
pop[1].printConnectGenes()
print('')
pop[1].nodeGenes[1]['adjacentNodes']
pop[1].species

{1: {'type': 'input', 'value': 1.0, 'valueHistory': [], 'adjacentNodes': []}, 2: {'type': 'input', 'value': 0.016671586501195407, 'valueHistory': [], 'adjacentNodes': []}, 3: {'type': 'input', 'value': -0.0011039398650791593, 'valueHistory': [], 'adjacentNodes': []}, 4: {'type': 'input', 'value': 0.010957962710434935, 'valueHistory': [], 'adjacentNodes': []}, 5: {'type': 'input', 'value': -0.019693178235587763, 'valueHistory': [], 'adjacentNodes': []}, 6: {'type': 'output', 'value': -2.0, 'valueHistory': [], 'adjacentNodes': [1, 2, 3, 4, 5, 8]}, 7: {'type': 'output', 'value': -2.0, 'valueHistory': [], 'adjacentNodes': [1, 2, 3, 4, 5]}}

{(1, 6): {'historicalMarker': 1, 'weight': 0.30934010097235265, 'isExpressed': True}, (1, 7): {'historicalMarker': 2, 'weight': 0.34207247516054906, 'isExpressed': True}, (2, 6): {'historicalMarker': 3, 'weight': -0.9282397629602213, 'isExpressed': True}, (2, 7): {'historicalMarker': 4, 'weight': -0.35460012929990015, 'isExpressed': True}, (3, 6): {'his

1

In [ ]:
for n in pop[1].nodeGenes:
    print(pop[1].nodeGenes[n]['adjacentNodes'])

In [ ]:
# load the networks that solved the problem !
with open('data/bestGenome_cartpole.dictionary', 'rb') as dictionary_file:
    bestGenomeDictionary = pickle.load(dictionary_file)

In [ ]:
testList = [1, 3, 5, 2, 0, -1, 2, 3]

In [ ]:
testList.sort()

In [ ]:
print(testList)

In [ ]:
species[s]['list'].sort(reverse = True, key=lambda genome: genome.fitness)

In [ ]:
for genome in species[s]['list']:
    print(genome.fitness)

In [ ]:
# test bench 

env = gym.make(envName)

testBenchTotalFitness = 0

for i in range(100):
    t = 0
    bestGenome.resetFitness()
    observation = env.reset()
    while(True):
        t+=1
        env.render()
        action = bestGenome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        bestGenome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            testBenchTotalFitness+=bestGenome.fitness
            print("Genome fitness: {}".format(bestGenome.fitness))
            break

print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/100))
env.close()